<a href="https://colab.research.google.com/github/kebreck/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/U1_Build_Breckenfelder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [1]:
!wget 'https://github.com/the-trace-and-buzzfeed-news/local-police-data-analysis/blob/master/inputs/offenses-standardized.csv.zip'

--2020-06-26 04:12:14--  https://github.com/the-trace-and-buzzfeed-news/local-police-data-analysis/blob/master/inputs/offenses-standardized.csv.zip
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘offenses-standardized.csv.zip’

offenses-standardiz     [ <=>                ]  73.32K  --.-KB/s    in 0.01s   

2020-06-26 04:12:14 (6.24 MB/s) - ‘offenses-standardized.csv.zip’ saved [75077]



In [13]:
! unzip 'offenses-standardized.csv.zip'

Archive:  offenses-standardized.csv.zip
  inflating: offenses-standardized.csv  


In [4]:
crimes = pd.read_csv('offenses-standardized.csv')

crimes.sample(10)

FileNotFoundError: ignored

In [42]:
crimes_small = crimes[['offense_group', 'clearance_status',]].copy()

crimes_small.sample(20)

,offense_group,clearance_status
128277,Agg Assault - Other Weapon,NaN
14964,Agg Assault - Other Weapon,NaN
575516,Agg Assault - Other Weapon,NaN
279520,Homicide - Gun,OPEN
588675,Agg Assault - Other Weapon,NaN
154768,Agg Assault - Gun,NaN
65577,Agg Assault - Other Weapon,NaN
118914,Agg Assault - Gun,NaN
128634,Agg Assault - Other Weapon,NaN
631022,Agg Assault - Other Weapon,CLEARED-EX


In [15]:
crimes['offense_group'].value_counts()

Agg Assault - Other Weapon          424888
Agg Assault - Gun                   270227
Homicide - Gun                       25212
Agg Assault - Weapon Unspecified     11500
Non-Fatal Shooting                    8284
Homicide - Other Weapon               6975
Homicide - Weapon Unspecified          572
Name: offense_group, dtype: int64

In [25]:
#Bar chart of crime types

# crimes['offense_group'].value_counts().plot(kind='bar', figsize=(12,12));

In [55]:
crimes['clearance_status'].value_counts()

OPEN                   183907
CLEARED-ARREST         101787
CLEARED-EX              31345
CLEARED-UNSPECIFIED     22742
CLOSED-UNSPECIFIED      16505
UNCLEAR                   425
ARREST-UNSPECIFIED        294
MISSING                   247
UNKNOWN                     3
Name: clearance_status, dtype: int64

In [26]:
#Pie chart of clearance

# crimes['clearance_group'].value_counts().plot(kind='pie', figsize=(12,12));

In [78]:
homicide_categories = ['Homicide - Gun', 'Homicide - Other Weapon', 'Homicide - Weapon Unspecified']

# Agg Assault - Other Weapon          
# Agg Assault - Gun                   
# Homicide - Gun                       
# Agg Assault - Weapon Unspecified     
# Non-Fatal Shooting                    
# Homicide - Other Weapon               
# Homicide - Weapon Unspecified

def group_offenses(offense_group):
  if offense_group in homicide_categories:
    return 'Homicide'
  else:
    return "Assault"

In [79]:
clearance_categories = ['UNCLEAR', 'MISSING', 'UNKNOWN']

# OPEN                   
# CLEARED-ARREST         
# CLEARED-EX             
# CLEARED-UNSPECIFIED     
# CLOSED-UNSPECIFIED      
# UNCLEAR                   
# ARREST-UNSPECIFIED        
# MISSING                   
# UNKNOWN                     

def group_clearance(clearance_status):
  if clearance_status in clearance_categories:
    return 'UNKNOWN'
  elif clearance_status == 'OPEN':
    return 'OPEN'
  else:
    return 'CLEARED'

In [80]:
crimes['clearance_status'][279520]

'OPEN'

In [82]:
group_clearance(crimes['clearance_status'][279520])

'OPEN'

In [84]:
crimes_small['offense'] = crimes_small['offense_group'].apply(group_offenses)

crimes_small['offense'].value_counts()

Assault     714899
Homicide     32759
Name: offense, dtype: int64

In [85]:
crimes_small['clearance'] = crimes_small['clearance_status'].apply(group_clearance)

crimes_small['clearance'].value_counts()

CLEARED    563076
OPEN       183907
UNKNOWN       675
Name: clearance, dtype: int64

In [125]:
# crimes_small.plot(kind = 'scatter', x = 'arrest_date', y = 'victim_age', c='Pclass', colormap='plasma', figsize=(15,4));

In [87]:
assaults = crimes_small[crimes_small['offense']=='Assault']

assaults.head()

,offense_group,clearance_status,offense,clearance
0,Non-Fatal Shooting,OPEN,Assault,OPEN
1,Non-Fatal Shooting,OPEN,Assault,OPEN
2,Non-Fatal Shooting,OPEN,Assault,OPEN
3,Non-Fatal Shooting,OPEN,Assault,OPEN
4,Non-Fatal Shooting,OPEN,Assault,OPEN


In [88]:
homicides = crimes_small[crimes_small['offense']=='Homicide']

homicides.head()

,offense_group,clearance_status,offense,clearance
3621,Homicide - Gun,OPEN,Homicide,OPEN
3622,Homicide - Gun,OPEN,Homicide,OPEN
3623,Homicide - Gun,OPEN,Homicide,OPEN
3624,Homicide - Gun,OPEN,Homicide,OPEN
3625,Homicide - Other Weapon,OPEN,Homicide,OPEN


In [92]:
# Homicide Clearance Rate

homicide_xtab = pd.crosstab(homicides['clearance'], homicides['offense'], 
            normalize=True,
            margins=True)

homicide_xtab['Homicide']['CLEARED']

0.6062150859305839

In [94]:
# Assault Clearance Rate

assault_xtab = pd.crosstab(assaults['clearance'], assaults['offense'], 
            normalize=True,
            margins=True)

assault_xtab['Assault']['CLEARED']

0.7598513915951763

# Charts

In [116]:
homicides['clearance'].value_counts()

CLEARED    19859
OPEN       12667
UNKNOWN      233
Name: clearance, dtype: int64

In [122]:
import plotly.graph_objects as go

labels = ['Cleared','Open','Unknown']
values = homicides['clearance'].value_counts()

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels, 
                             values=values, 
                             pull=[0, 0.3, 0])])
fig.show()

In [126]:
labels = ['Cleared','Open','Unknown']
values = assaults['clearance'].value_counts()

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels, 
                             values=values, 
                             pull=[0, 0.3, 0])])
fig.show()